In [50]:
import pandas as pd
from datetime import datetime


In [51]:
# Get data Function
# Written as functions for testing in the future and if the source changes the function calls are still valid
def get_enrollment_data() -> pd.DataFrame:
    data = pd.read_excel("enrollments.xlsx")
    return pd.DataFrame(data)

def get_gdp_data() -> pd.DataFrame:
    data = pd.read_csv("Countries_GDP_1960-2020.csv")
    return pd.DataFrame(data)

In [52]:
#  Validate Schema
def validate_enrollment_schema(data: pd.DataFrame) -> bool:
    # We want to check that the column names are what we are expecting
    # if the file changes we need to adjust this validation
    valid_columns = [
        "country",
        "countrycode",
        "region",
        "incomegroup",
        "iau_id",
        "iau_id1",
        "eng_name",
        "orig_name",
        "foundedyr",
        "yrclosed",
        "private01",
        "coordinates",
        "latitude",
        "longitude",
        "phd_granting",
        "m_granting",
        "b_granting",
        "divisions",
        "total_fields",
        "unique_fields",
        "specialized",
        "merger",
        "noiau",
        "year",
        "students5_interpolated",
        "students5_extrapolated",
        "students5_estimated"
    ]
    columns = data.columns
    if set(columns) == set(valid_columns):
        return True
    return False

def validate_gdp_schema(data: pd.DataFrame) -> bool:
    # The dates for valid schema will change as more data is added
    # Country Name and Country Code are enough for this file
    # we really don't need to update this function every year
    valid_columns = [
        "Country Name",
        "Country Code"
    ]
    columns = data.columns
    valid_columns_exist = 0
    for valid_column in valid_columns:
        for column in columns:
            if column == valid_column:
                valid_columns_exist += 1
    print(valid_columns_exist)
    if valid_columns_exist == len(valid_columns):
        return True
    return False


In [53]:
# Enrollment Transformations
def enrollment_convert_types(df: pd.DataFrame) -> pd.DataFrame:
    # Convert items to int
    # Fill NA values with 0
    #   Making an assumption that 0 is true and 1 is false to move forward with the exercise
    #   I would raise assumption to stakeholders making sure my assumption is correct and the best use for this data
    #   In this exercise I am just going ahead because it is an exercise
    numeric_columns = {
        'private01': 'int32',
        'm_granting': 'int32',
        'b_granting': 'int32',
        'divisions': 'int32',
        'total_fields': 'int32',
        'unique_fields': 'int32',
        'merger': 'int32',
        'noiau': 'int32',
        'students5_interpolated': 'int32',
        'students5_extrapolated': 'int32',
        'students5_estimated': 'int32',
        "foundedyr": 'int32',
        'year': 'int32',
    }
    # Removed Lat and Long form typing I need to be finishing this exercise up to stay close to the limits
    # Hurts me deply to not finish that but got to go fast right now

    fill_na_column_names = list(numeric_columns)
    df[fill_na_column_names] = df[fill_na_column_names].fillna(0)

    df = df.astype(numeric_columns)
    string_columns = {
        'country': 'string',
        'countrycode': 'string',
        'region': 'string',
        'incomegroup': 'string',
        'iau_id': 'string',
        'iau_id1': 'string',
        'eng_name': 'string',
        'orig_name': 'string',
        'coordinates': 'string'
    }
    df = df.astype(string_columns)
    return df


In [54]:
# GDP Transformation
def transform_GDP(df: pd.DataFrame) -> pd.DataFrame:
    # un-pivoting the data to be country|country code| year
    # this will allow us to join to the enrollment data easier
    df = pd.melt(
        df,
        id_vars=['Country Name', 'Country Code'],
        var_name="Year",
        value_name="GDP"
    )
    convert_columns = {
        'Country Name': 'string',
        'Country Code': 'string',
        'Year': 'int32'
    }
    df = df.astype(convert_columns)
    df = df.rename(columns={
        'Country Name': 'Country_Name',
        'Country Code': 'Country_Code',
    })
    return df


In [55]:
def merge_data(gdp: pd.DataFrame, enrollment: pd.DataFrame) -> pd.DataFrame:
    df = enrollment.merge(
        gdp,
        how='left',
        left_on=['year', 'countrycode'],
        right_on=['Year', 'Country_Code']
    )
    return df

In [56]:
df_gdp = get_gdp_data()
df_enrollment = get_enrollment_data()

In [57]:
df_gdp_clean = transform_GDP(df_gdp)
df_enrollment_clean = enrollment_convert_types(df_enrollment)

In [58]:
df_final = merge_data(df_gdp_clean, df_enrollment_clean)
df_final.sample(10)

,country,countrycode,region,incomegroup,iau_id,iau_id1,eng_name,orig_name,foundedyr,yrclosed,...,merger,noiau,year,students5_interpolated,students5_extrapolated,students5_estimated,Country_Name,Country_Code,Year,GDP
13144,brazil,BRA,Latin America and Caribbean,Upper middle income,IAU-018562,IAU-018562-1,University Of Santa Cruz Do Sul,<NA>,1964,NaN,...,0,0,1995,0,6284,6284,<NA>,<NA>,NaN,NaN
158425,united states,USA,North America,High income,IAU-026051��,IAU-026051��-1,Bryan University - Tempe,<NA>,1940,NaN,...,0,0,2020,0,2485,2485,United States,USA,2020.0,2.100000e+13
112564,russian federation,RUS,Europe and Central Asia,Upper middle income,IAU-003868,IAU-003868-2,Don State Technical University,Donskoj Gosudarstvennyj Tekhni��eskij Universitet,1930,NaN,...,1,0,1975,0,4019,0,<NA>,<NA>,NaN,NaN
48515,ghana,GHA,Sub-Saharan Africa,Lower middle income,IAU-024262,IAU-024262-1,Christ Apostolic University College,Christ Apostolic University College,1917,NaN,...,0,0,1955,0,0,0,<NA>,<NA>,NaN,NaN
8623,brazil,BRA,Latin America and Caribbean,Upper middle income,IAU-002598,IAU-002598-1,Salesian University Center Of S��O Paulo,Centro Universit��rio Salesiano de S��o Paulo ...,1952,NaN,...,0,0,1955,0,0,0,<NA>,<NA>,NaN,NaN
107630,poland,POL,Europe and Central Asia,High income,IAU-013073,IAU-013073-1,J��Zef Rusiecki Olsztyn Higher School,Olszty��ska Szko��a Wy��sza im. J��zefa Rusiec...,1997,NaN,...,0,0,2015,0,0,1890,<NA>,<NA>,NaN,NaN
134093,ukraine,UKR,Europe and Central Asia,Lower middle income,IAU-026770,IAU-026770-1,Ukrainian State Employment Service Training In...,Instytut pidhotovky kadriv der��avnoji slu��by...,1993,NaN,...,0,0,1995,0,0,2551,<NA>,<NA>,NaN,NaN
152232,united states,USA,North America,High income,IAU-015930,IAU-015930-1,On The Ross State University,<NA>,1917,NaN,...,0,0,1975,2796,2796,2796,United States,USA,1975.0,1.680000e+12
63784,iran,IRN,Middle East and North Africa,Upper middle income,IAU-015464,IAU-015464-1,Sistan And Baluchistan University,(USB),1974,NaN,...,0,0,1985,1260,1260,1260,<NA>,<NA>,NaN,NaN
15521,bulgaria,BGR,Europe and Central Asia,Upper middle income,IAU-019871,IAU-019871-1,Prof. D-R Assen Zlatarov University - Burgas,Burgas Prof. Assen Zlatarov University (BTU),1963,NaN,...,0,0,1970,2254,2254,2254,<NA>,<NA>,NaN,NaN


In [59]:
pd.set_option("display.max_info_columns", 30)
df_final.info(verbose=True)
pd.reset_option("display.max_info_columns")
# Not 100% happy with Year being a float64, but I need to wrap up to stay close to time

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161560 entries, 0 to 161559
Data columns (total 31 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   country                 string 
 1   countrycode             string 
 2   region                  string 
 3   incomegroup             string 
 4   iau_id                  string 
 5   iau_id1                 string 
 6   eng_name                string 
 7   orig_name               string 
 8   foundedyr               int32  
 9   yrclosed                float64
 10  private01               int32  
 11  coordinates             string 
 12  latitude                object 
 13  longitude               object 
 14  phd_granting            float64
 15  m_granting              int32  
 16  b_granting              int32  
 17  divisions               int32  
 18  total_fields            int32  
 19  unique_fields           int32  
 20  specialized             float64
 21  merger                  int32  
 

In [60]:
# Validation checks
# This type of join should never have more than 1 enrollment record or else we have a cardinality violation
assert df_final.shape[0] <= (df_gdp_clean.shape[0] + df_enrollment_clean.shape[0])

# Make sure that all post 1960 country codes that are represented in GDP are matched
df_missing_country_codes = df_final.query("Country_Code.isnull() and GDP.isnull() and year > 1960", engine='python')
missing_country_codes = df_missing_country_codes['countrycode'].unique()
expected_country_codes = df_gdp_clean['Country_Code'].unique()
assert len(list(set(expected_country_codes) & set(missing_country_codes))) == 0

In [61]:
df_final.to_csv("final_data.csv", index=False)